In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("../GTZAN/Data/features_30_sec.csv")
df = data.sample(frac=1).reset_index(drop=True)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,rock.00045.wav,661794,0.361234,0.082603,0.096995,0.000815,1560.999792,90382.491401,1568.308311,10787.185064,...,25.971584,-11.281557,17.765968,-7.829482,20.357563,-7.715235,21.045958,-8.141011,21.702080,rock
1,reggae.00041.wav,661504,0.373656,0.091907,0.119993,0.000973,1930.232326,589491.544262,2204.904559,155397.743357,...,49.393574,-1.406729,81.619194,-1.651171,53.031818,1.212168,56.808544,-1.888977,54.328857,reggae
2,rock.00035.wav,661794,0.457206,0.079384,0.124158,0.000459,1867.082394,398470.110770,2422.014649,281438.579580,...,24.703766,-0.031292,29.246223,-1.009292,24.503607,-5.568185,31.999117,-0.845786,34.502522,rock
3,rock.00094.wav,661794,0.389313,0.073420,0.101960,0.000372,2628.863788,93936.455555,2210.478936,37253.811076,...,26.217716,-12.762190,24.685890,2.999796,31.530489,-6.408553,31.573473,2.254876,38.511021,rock
4,blues.00006.wav,661794,0.291328,0.093981,0.141874,0.008803,1459.366472,437859.398051,1389.009131,185023.239545,...,83.248245,-10.913176,56.902153,-6.971336,38.231800,-3.436505,48.235741,-6.483466,70.170364,blues


In [3]:
dummy = df.drop(['filename', 'length'], axis=1)
column_name = 'label'
label_encoder = LabelEncoder()
dummy[column_name] = label_encoder.fit_transform(dummy[column_name])
corr = dummy.corr()
corr_style = corr.style.background_gradient(cmap='coolwarm')
display(corr_style)

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
chroma_stft_mean,1.000000,-0.461729,0.499926,0.261544,0.615728,0.285926,0.517710,0.041088,0.598122,0.117139,0.554771,0.355920,0.031963,0.281150,-0.116567,0.465995,0.018477,0.659550,-0.125173,-0.598713,-0.052177,-0.054353,0.003765,0.264147,0.132253,-0.010085,0.038402,0.416877,0.101364,0.067508,0.066712,0.509776,-0.026696,0.116040,-0.151147,0.489229,-0.192695,0.072779,-0.301768,0.516690,-0.408530,-0.081445,-0.438212,0.417091,-0.444702,-0.143372,-0.440796,0.332359,-0.452955,-0.163474,-0.456298,0.269935,-0.478452,-0.089509,-0.501768,0.281965,-0.548577,0.366451
chroma_stft_var,-0.461729,1.000000,-0.078777,0.290138,-0.246441,0.305819,-0.036896,0.346907,-0.168721,0.383134,-0.470186,0.169289,-0.006378,0.047447,0.086584,0.020981,-0.079209,-0.343372,0.367570,0.118888,0.353811,0.412857,0.421817,-0.252334,0.394627,0.354068,0.414076,-0.209684,0.447260,0.277560,0.433032,-0.318189,0.450482,0.206969,0.469733,-0.292359,0.421070,0.231738,0.395838,-0.359185,0.382765,0.314236,0.383944,-0.275502,0.365766,0.273446,0.372656,-0.311178,0.361752,0.205117,0.389468,-0.239597,0.378200,0.184323,0.365291,-0.231715,0.333581,-0.082835
rms_mean,0.499926,-0.078777,1.000000,0.607399,0.510154,0.324077,0.530017,0.099830,0.537902,0.200083,0.332163,0.319304,0.064440,0.893898,-0.009166,0.789994,-0.052634,0.807239,-0.179467,-0.484057,-0.009069,0.139513,-0.024116,0.008731,0.180446,0.108939,0.068321,0.125682,0.213669,0.167750,0.114015,0.178947,0.083730,0.216109,0.011487,0.186127,-0.005768,0.117318,-0.118258,0.183280,-0.195578,0.013036,-0.236572,0.163009,-0.216350,-0.011154,-0.214653,0.083936,-0.160347,-0.029562,-0.163490,0.118239,-0.164834,0.031297,-0.207680,0.124413,-0.252434,0.213968
rms_var,0.261544,0.290138,0.607399,1.000000,0.347831,0.583353,0.402333,0.320793,0.368570,0.417880,0.156530,0.568643,0.084849,0.585318,0.123557,0.789910,-0.118671,0.326590,0.276735,-0.368384,0.348277,0.297067,0.351018,-0.234197,0.465319,0.265403,0.359638,-0.155544,0.533171,0.312285,0.357660,-0.126337,0.407201,0.328440,0.329269,-0.071458,0.287186,0.266139,0.180124,-0.077560,0.106692,0.229806,0.099020,-0.037588,0.059604,0.211396,0.062595,-0.093893,0.091863,0.168169,0.100234,-0.002585,0.058050,0.186953,0.049879,0.029364,0.012087,0.107572
spectral_centroid_mean,0.615728,-0.246441,0.510154,0.347831,1.000000,0.520548,0.904435,0.011256,0.979637,0.198031,0.874749,0.671771,0.053695,0.316211,-0.013444,0.559647,-0.025764,0.721006,-0.201153,-0.940180,-0.017894,0.188000,-0.099545,-0.136810,0.153690,0.069132,-0.078022,0.017597,0.171060,0.215637,0.024782,0.134407,0.051892,0.288884,-0.066127,0.192896,-0.069192,0.152689,-0.143950,0.175123,-0.234021,0.051308,-0.220678,0.170910,-0.203427,0.022383,-0.204568,0.153741,-0.180200,0.027473,-0.158666,0.228275,-0.136272,0.027918,-0.146316,0.269601,-0.205531,0.378395
spectral_centroid_var,0.285926,0.305819,0.324077,0.583353,0.520548,1.000000,0.622834,0.636494,0.545128,0.815550,0.252065,0.844141,0.068491,0.276994,0.133146,0.485951,-0.100085,0.209955,0.356401,-0.546578,0.683143,0.561375,0.400939,-0.358388,0.568875,0.466334,0.404730,-0.216838,0.562846,0.480165,0.427062,-0.199376,0.436999,0.463182,0.370555,-0.170516,0.334639,0.382478,0.190391,-0.176640,0.107455,0.366881,0.103422,-0.150271,0.070390,0.304984,0.055370,-0.186864,0.072319,0.175573,0.094155,-0.063059,0.

In [4]:
label_encoder = LabelEncoder()
target = "label"
x = df.drop(['filename', 'length',target], axis=1)
y = df[target]
y= label_encoder.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, stratify=y, random_state=100)
y

array([9, 8, 9, 9, 0, 4, 7, 7, 3, 5, 0, 7, 5, 9, 0, 8, 1, 5, 4, 5, 4, 6,
       3, 8, 1, 8, 1, 8, 8, 8, 6, 1, 8, 5, 6, 0, 7, 0, 7, 5, 8, 6, 4, 9,
       9, 9, 8, 0, 1, 3, 4, 8, 5, 9, 0, 1, 7, 1, 0, 4, 9, 7, 3, 5, 3, 3,
       2, 9, 4, 8, 8, 6, 8, 9, 5, 9, 4, 7, 5, 6, 6, 1, 9, 2, 4, 0, 4, 0,
       0, 9, 4, 4, 7, 3, 6, 7, 9, 7, 7, 2, 8, 3, 9, 6, 4, 9, 1, 9, 8, 1,
       9, 5, 1, 6, 2, 7, 1, 6, 3, 5, 2, 3, 8, 4, 4, 6, 4, 6, 5, 4, 3, 4,
       7, 6, 5, 3, 7, 7, 4, 2, 5, 6, 7, 2, 1, 1, 2, 5, 2, 0, 1, 6, 0, 2,
       8, 0, 4, 3, 0, 5, 4, 2, 1, 0, 0, 1, 2, 9, 2, 5, 6, 9, 4, 5, 1, 7,
       5, 9, 3, 3, 9, 8, 6, 5, 7, 7, 6, 3, 7, 7, 5, 1, 0, 9, 7, 7, 3, 8,
       6, 0, 8, 6, 7, 4, 4, 2, 5, 0, 7, 3, 8, 7, 1, 3, 4, 0, 3, 0, 1, 5,
       1, 1, 5, 0, 7, 6, 8, 1, 3, 1, 4, 8, 1, 8, 7, 2, 1, 4, 0, 6, 8, 5,
       7, 0, 8, 0, 9, 3, 0, 7, 5, 0, 0, 8, 7, 9, 6, 3, 0, 7, 0, 9, 6, 3,
       3, 0, 3, 0, 0, 2, 2, 6, 3, 9, 6, 1, 7, 5, 9, 6, 9, 6, 0, 3, 3, 0,
       3, 2, 1, 9, 3, 8, 8, 3, 8, 9, 7, 8, 0, 7, 1,

In [5]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train

array([[ 0.08239042,  0.69742594, -0.61428834, ...,  0.10873433,
        -0.39380116,  0.34253897],
       [ 0.90778593, -0.42928155, -0.55685707, ..., -0.46936952,
        -1.40352797, -0.11038481],
       [-2.01849352, -0.23136227, -1.33232802, ...,  0.95385817,
         0.61289771,  1.81522969],
       ...,
       [ 0.86700697,  0.54063469,  1.81830724, ...,  1.56494688,
         0.9834829 ,  0.9088723 ],
       [-0.98477293, -0.06645381, -0.64898999, ..., -0.06100823,
        -0.20127231,  0.31504643],
       [ 0.76049484, -1.05265172,  0.28366503, ..., -0.87753751,
         0.61010253, -0.66633846]])

In [6]:
model = LGBMClassifier(boosting_type= 'gbdt', learning_rate=0.2, max_depth= 7, num_leaves=150)
model.fit(x_train,y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003769 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14313
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 57
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

LGBMClassifier(learning_rate=0.2, max_depth=7, num_leaves=150)

In [7]:
predictions = model.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))
print(classification_report(y_test, predictions))

Accuracy: 72.00%
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        20
           1       0.90      0.90      0.90        20
           2       0.64      0.80      0.71        20
           3       0.67      0.40      0.50        20
           4       0.60      0.75      0.67        20
           5       0.89      0.85      0.87        20
           6       0.94      0.85      0.89        20
           7       0.74      0.70      0.72        20
           8       0.57      0.65      0.60        20
           9       0.47      0.45      0.46        20

    accuracy                           0.72       200
   macro avg       0.73      0.72      0.72       200
weighted avg       0.73      0.72      0.72       200



In [8]:
cm=confusion_matrix(y_test, predictions)
category_names = ["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
true_positives = np.diag(cm)
false_positives = np.sum(cm, axis=0) - true_positives
false_negatives = np.sum(cm, axis=1) - true_positives
accuracy_per_label = true_positives / (true_positives + false_negatives)
print(confusion_matrix(y_test, predictions))
for i, accuracy in enumerate(accuracy_per_label):
    category_name = category_names[i]
    print("Accuracy for {}: {:.4f}".format(category_name, accuracy))

[[17  0  1  0  0  0  0  0  1  1]
 [ 0 18  0  0  0  1  0  0  0  1]
 [ 1  0 16  0  0  1  0  0  1  1]
 [ 0  1  0  8  6  0  0  4  0  1]
 [ 1  0  1  1 15  0  0  0  2  0]
 [ 0  1  0  0  0 17  0  0  0  2]
 [ 1  0  0  0  0  0 17  0  0  2]
 [ 0  0  2  0  1  0  0 14  3  0]
 [ 0  0  2  1  1  0  0  1 13  2]
 [ 0  0  3  2  2  0  1  0  3  9]]
Accuracy for blues: 0.8500
Accuracy for classical: 0.9000
Accuracy for country: 0.8000
Accuracy for disco: 0.4000
Accuracy for hiphop: 0.7500
Accuracy for jazz: 0.8500
Accuracy for metal: 0.8500
Accuracy for pop: 0.7000
Accuracy for reggae: 0.6500
Accuracy for rock: 0.4500
